In [1]:
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class mnistmodel_A2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=5,stride=2)
        self.dense1 = nn.Linear(in_features=64*12*12,out_features=32)
        self.dense2 = nn.Linear(in_features=32,out_features=2)
        self.dense3 = nn.Linear(in_features=2,out_features=1)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        #x = F.max_pool2d(x,2)
        x = F.relu(self.conv2(x))
        #x = F.max_pool2d(x,2)
        x = F.dropout(x,0.25)
        x = x.view(-1,64*12*12)
        x = F.sigmoid(self.dense1(x))
        x = F.dropout(x,0.5)
        x = F.sigmoid(self.dense2(x))
        x = F.sigmoid(self.dense3(x))

        return x
    

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
from torchsummary import summary

In [5]:
summary(mnistmodel_A2().to(device),(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]           1,664
            Conv2d-2           [-1, 64, 12, 12]         102,464
            Linear-3                   [-1, 32]         294,944
            Linear-4                    [-1, 2]              66
            Linear-5                    [-1, 1]               3
Total params: 399,141
Trainable params: 399,141
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.45
Params size (MB): 1.52
Estimated Total Size (MB): 1.98
----------------------------------------------------------------


/home/ml-lab/anaconda3/envs/ptorch/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [6]:
model = mnistmodel_A2().to(device)

In [7]:
import getData_pt as getData

In [8]:
train_data = getData.normalMnist(data_type='train')
test_data = getData.normalMnist(data_type='test')

In [9]:
loder_train = train_data.loader
loder_test = test_data.loader

In [10]:
train_losses = []
train_counter = []
test_losses = []
n_epochs = 20
test_counter = [i*len(loder_train.dataset) for i in range(n_epochs + 1)]

In [11]:
from tqdm.notebook import tqdm

In [12]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=5,verbose=True,factor=0.5)
log_interval = 10
#cel_loss = nn.CrossEntropyLoss()
cel_loss = nn.BCELoss()
def train(epoch):
    #bbar =  tqdm(total= len(loder_train),desc="Batch")
    model.train()
    for batch_idx, (data, target) in enumerate(loder_train):
        data = data.to(device)
        target = target.to(device).type(torch.float)
        optimizer.zero_grad()
        output = model(data)
        loss = cel_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            '''print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loder_train.dataset),
                100. * batch_idx / len(loder_train), loss.item()))'''
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(loder_train.dataset)))
            #torch.save(network.state_dict(), './results/model.pth')
            #torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [13]:
def test(ret=False):
    model.eval()
    #torch.manual_seed(999)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loder_test:
            data = data.to(device)
            target = target.to(device).type(torch.float)
            #print(data.shape)
            output = model(data)
            #print(output.shape,target.shape)
            test_loss += F.binary_cross_entropy(output, target).item()
            #pred = output.data.max(1, keepdim=True)[1]
            pred = (output>0.5).type(torch.int)
            correct += (pred==target).sum()
        test_loss /= len(loder_test.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(loder_test.dataset),100. * correct / len(loder_test.dataset)))
        if ret == True:
            return (float(correct) / len(loder_test.dataset))

In [14]:
test(ret=True)


Test set: Avg. loss: 0.0059, Accuracy: 1135/2115 (53.66%)



0.5366430260047281

In [15]:
for epoch in tqdm(range(1, n_epochs + 1)):
    train(epoch)
    #test()
    scheduler.step(test(ret=True))


Test set: Avg. loss: 0.0050, Accuracy: 1135/2115 (53.66%)


Test set: Avg. loss: 0.0044, Accuracy: 1135/2115 (53.66%)


Test set: Avg. loss: 0.0040, Accuracy: 1537/2115 (72.67%)


Test set: Avg. loss: 0.0036, Accuracy: 2092/2115 (98.91%)


Test set: Avg. loss: 0.0033, Accuracy: 2108/2115 (99.67%)


Test set: Avg. loss: 0.0030, Accuracy: 2112/2115 (99.86%)


Test set: Avg. loss: 0.0028, Accuracy: 2113/2115 (99.91%)

Epoch     7: reducing learning rate of group 0 to 5.0000e-04.

Test set: Avg. loss: 0.0027, Accuracy: 2112/2115 (99.86%)


Test set: Avg. loss: 0.0026, Accuracy: 2113/2115 (99.91%)


Test set: Avg. loss: 0.0025, Accuracy: 2113/2115 (99.91%)


Test set: Avg. loss: 0.0024, Accuracy: 2113/2115 (99.91%)


Test set: Avg. loss: 0.0023, Accuracy: 2113/2115 (99.91%)


Test set: Avg. loss: 0.0022, Accuracy: 2111/2115 (99.81%)

Epoch    13: reducing learning rate of group 0 to 2.5000e-04.

Test set: Avg. loss: 0.0022, Accuracy: 2113/2115 (99.91%)


Test set: Avg. loss: 0.0021, Accura

In [16]:
test()


Test set: Avg. loss: 0.0020, Accuracy: 2113/2115 (99.91%)



In [17]:
atk_data = getData.attackMnist(model,atk_loss=nn.BCELoss())

In [18]:
def test(data_loader,ret=False):
    loder_test = data_loader
    model.eval()
    #torch.manual_seed(999)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loder_test:
            data = data.to(device)
            target = target.to(device).type(torch.float)
            #print(data.shape)
            output = model(data)
            #print(output.shape,target.shape)
            test_loss += F.binary_cross_entropy(output, target).item()
            #pred = output.data.max(1, keepdim=True)[1]
            pred = (output>0.5).type(torch.int)
            correct += (pred==target).sum()
        test_loss /= len(loder_test.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(loder_test.dataset),100. * correct / len(loder_test.dataset)))
        if ret == True:
            return (float(correct) / len(loder_test.dataset))

In [19]:
test(atk_data.loader)


Test set: Avg. loss: 0.0035, Accuracy: 1787/2115 (84.49%)



In [20]:
atk_data.loader

In [21]:
torch.save(model,'mnist_model/modelA2.pth')

/home/ml-lab/anaconda3/envs/ptorch/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type mnistmodel_A2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
